<a href="https://colab.research.google.com/github/Aparimita18/Question_Answering_Model/blob/main/QA_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install PyMuPDF transformers gradio

In [21]:
import fitz

book = fitz.open("/content/dataset.pdf")

# Extract text from all pages
data = ""
for page in book:
    data += page.get_text()

print(data[:1000])

Speech and Language Processing
An Introduction to Natural Language Processing,
Computational Linguistics, and Speech Recognition
with Language Models
Third Edition draft
Daniel Jurafsky
Stanford University
James H. Martin
University of Colorado at Boulder
Copyright ©2024. All rights reserved.
Draft of January 12, 2025. Comments and typos welcome!
Summary of Contents
I
Fundamental Algorithms for NLP
1
1
Introduction. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
3
2
Regular Expressions, Tokenization, Edit Distance . . . . . . . . . . . . . . .
4
3
N-gram Language Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
32
4
Naive Bayes, Text Classiﬁcation, and Sentiment . . . . . . . . . . . . . . . . . 56
5
Logistic Regression . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 77
6
Vector Semantics and Embeddings . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 10

In [22]:
import fitz  # PyMuPDF
import pandas as pd

# Open PDF
doc = fitz.open("dataset.pdf")
page = doc[0]  # First page

# returns list of dicts with positional info
blocks = page.get_text("dict")["blocks"]

# Filter for text blocks
table_data = []
for b in blocks:
    if b["type"] == 0:  # text
        for line in b["lines"]:
            row = []
            for span in line["spans"]:
                row.append(span["text"].strip())
            if row:
                table_data.append(row)

# convert to DataFrame
for row in table_data:
    print(row)

df = pd.DataFrame(table_data)
print(df)


['Speech and Language Processing']
['An Introduction to Natural Language Processing,']
['Computational Linguistics, and Speech Recognition']
['with Language Models']
['Third Edition draft']
['Daniel Jurafsky']
['Stanford University']
['James H. Martin']
['University of Colorado at Boulder']
['Copyright ©2024. All rights reserved.']
['Draft of January 12, 2025. Comments and typos welcome!']
                                                    0
0                      Speech and Language Processing
1     An Introduction to Natural Language Processing,
2   Computational Linguistics, and Speech Recognition
3                                with Language Models
4                                 Third Edition draft
5                                     Daniel Jurafsky
6                                 Stanford University
7                                     James H. Martin
8                   University of Colorado at Boulder
9               Copyright ©2024. All rights reserved.
10  Draft of 

In [23]:
import re

def clean(text):
    text = re.sub(r'-\s*\n\s*', '', text)
    text = re.sub(r'\n{2,}', '\n', text)  # Collapse multiple newlines
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r"-\n", "", text)


    text = re.sub(r'(Fig(?:ure)?\.? ?\d+(\.\d+)?(?:[^\w\n]{0,20})?)', '', text)
    text = re.sub(r"\(?[Tt]he interested reader.*?Section \d+(\.\d+)*.*?\)?", "", text)
    text = re.sub(r"\(?see (Section|Chapter|Subsection) \d+(\.\d+)*.*?\)?", "", text)
    text = re.sub(r"\(?refer to (Section|Chapter|Subsection) \d+(\.\d+)*.*?\)?", "", text)
    text = re.sub(r"\(?(as|as we) (see|saw) (in )?(Section|Chapter|Subsection) \d+(\.\d+)*.*?\)?", "", text)
    text = re.sub(r'^\s*[\*\-•]+.*$', '', text, flags=re.MULTILINE)
    #paragraphs = re.split(r'\n?(?=\d+(\.\d+)*\s+[^\n]+)', text)
    text = re.sub(r'^\s*\d+(\.\d+)*\s+[•\-–—]\s+.+?\d\s*$', '', text, flags=re.MULTILINE)

    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        line = line.strip()

        # Remove lines like "238 Chapter 8" or "Chapter 4"
        if re.match(r'^\d+\s+Chapter\s+\d+', line, re.IGNORECASE):
            continue
        if re.match(r'^Chapter\s+\d+', line, re.IGNORECASE):
            continue
        if re.match(r'^\d+\s+Chapter\s+\d+', line, re.IGNORECASE):
            continue
        if re.match(r'^Chapter\s+\d+', line, re.IGNORECASE):
            continue
        if re.fullmatch(r'\d+\s*[•\-–—]?', line) or re.fullmatch(r'[•\-–—]?\s*\d+', line):
            continue
        if re.fullmatch(r'\d+', line):
            continue
        if re.fullmatch(r'\d*\.\d+', line):
            continue
        if re.fullmatch(r'(Figure|Table|Section|Subsection)\s+\d+(\.\d+)*', line, re.IGNORECASE):
            continue
        if re.fullmatch(r'^•\s.*\s[\d.]+$',line):
            continue
        if re.search(r"(Speech and Language Processing|Jurafsky|Martin|Draft of \d{4})", line, re.IGNORECASE):
            continue
        if re.match(r'^[>\-•*]?\s*[A-Z\s]+$', line):
            continue
        if re.fullmatch(r'[\d\s•\-–—]*', line):
            continue
        if re.search(r"(C HAPTER|Speech and Language Processing|Jurafsky|Martin|Draft of \d{4}|CHAPTER \d+|N- GRAM|SPEECH RECOGNITION)", line, re.IGNORECASE):
            continue
        if re.fullmatch(r'[•\-–—]\s+[A-Z\s]{3,}', line) or line.isupper():
            continue
        if re.search(r'(Figure|Fig\.|Table|Box|Image|Diagram)\s*\d+(\.\d+)?', line, re.IGNORECASE):
            continue
        if re.fullmatch(r'[\[\(]?(image|diagram|fig|figure|illustration)[\]\)]?', line, re.IGNORECASE):
            continue
        if len(re.findall(r'[A-Za-z]', line)) < 4:
            continue
        if re.fullmatch(r'[•\-–—]\s+[A-Z\s]', line):
            continue

        cleaned_lines.append(line)

    cleaned_text = "\n".join(cleaned_lines)

    return cleaned_text


In [24]:
import re

def split_into_semantic_paragraphs(text):
    paragraphs = []
    current = ""

    for line in text.split("\n"):
        line = line.strip()

        # Skip empty lines only
        if not line:
           continue

        # If line looks like end of a sentence or paragraph, split
        if line.endswith(('.')) or re.match(r'.*\.[A-Z]', line):
            current += " " + line
            paragraphs.append(current.strip())
            current = ""
        else:
            current += " " + line

    # Add remaining paragraph
    if current.strip():
        paragraphs.append(current.strip())

    return paragraphs

In [25]:
def splitpara(text):
    paragraphs = []
    current = ""

# Define known footer patterns (can extend this as needed)
    footer_patterns = [
        r'^For example if we use both begin.*',
        r'^Speech and Language Processing.*',
        r'^Jurafsky.*',
        r'^Draft of \d{4}.*',
        r'^Page \d+.*',
        r'^Figure \d+.*',
        r'^.*\b\d{3,}\b.*$'  # Lines with page numbers
    ]


    cleaned_lines = []
    for line in text.split('\n'):
        line = line.strip()
        if any(re.match(pat, line) for pat in footer_patterns):
            continue
        cleaned_lines.append(line)


    '''# Clean up footers
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        if any(re.match(pat, line.strip()) for pat in footer_patterns):
            continue
        cleaned_lines.append(line)'''

    # Join back into cleaned text
    cleaned_text = '\n'.join(cleaned_lines)
    lines = cleaned_text.split('\n')

    # paragraph splitting based on subsection heading and punctuation
    #lines = cleaned_text.split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Remove figure/image/table mentions
        line = re.sub(r'(Fig(?:ure)?\.?\s*\d+(\.\d+)?(?:[^\w\n]{0,20})?)', '', line)
        line = re.sub(r'(Table\s*\d+(\.\d+)?(?:[^\w\n]{0,20})?)', '', line)
        line = re.sub(r'(Box\s*\d+(\.\d+)?(?:[^\w\n]{0,20})?)', '', line)
        line = re.sub(r'\(?(see|refer to)?\s*(Fig(?:ure)?|Table|Box)\s*\d+(\.\d+)?\)?', '', line, flags=re.IGNORECASE)


        if re.match(r'^\d+(\.\d+)+\s+[A-Za-z]', line):
            if current.strip():
                paragraphs.append(current.strip())
                current = ""
            paragraphs.append(line.strip())
            continue

        # If line ends with a period
        '''current += " " + line
        if line.endswith('.') or re.match(r'.*\.[A-Z]', line):
            #current += " " + line
            paragraphs.append(current.strip())
            current = ""'''

        # If line looks like end of a sentence or paragraph, split
        if line.endswith(('.')) or re.match(r'.*\.[A-Z]', line):
            current += " " + line
            paragraphs.append(current.strip())
            current = ""
        else:
            current += " " + line

        # Add remaining paragraph
        if current.strip():
            paragraphs.append(current.strip())

    '''final_paragraphs = []
    for para in paragraphs:
        sentences = sent_tokenize(para)
        merged = ''.join([sent.strip() for sent in sentences if sent.strip()])
        final_paragraphs.append(merged)

    return final_paragraphs'''

    #splitting on whitespace i.e 2 or more newlines
    final_paragraphs = []
    for para in paragraphs:
        chunks = re.split(r'\n\s*\n', para)
        final_paragraphs.extend([chunk.strip() for chunk in chunks if chunk.strip()])

    return final_paragraphs
    #return paragraphs'''

In [26]:


# Step 1: Clean logically
cleaned_text = clean(data)

# Step 2: Split meaningfully
paragraphs = splitpara(cleaned_text)

# Preview
import textwrap
print(f"Total paragraphs: {len(paragraphs)}")
print(textwrap.fill(paragraphs[1012]))


Total paragraphs: 25142
Word Normalization, Lemmatization and Stemming Word normalization is
the task of putting words or tokens in a standard format. The
normalization simplest case of word normalization is case folding.
Mapping everything to lower case folding case means that Woodchuck and
woodchuck are represented identically, which is very helpful for
generalization in many tasks, such as information retrieval or speech
recognition. For sentiment analysis and other text classiﬁcation
tasks, information


In [27]:
import pandas as pd

df = pd.DataFrame(paragraphs, columns=["text"])
df.to_csv("dataset.csv", index=False,sep="|", encoding="utf-8", header=True)
print("Saved as dataset.csv")

from google.colab import files
#files.download("dataset.csv")

Saved as dataset.csv


In [28]:
# Access a specific paragraph
print(textwrap.fill(df['text'][699]))

languages at the time of this writing, according to the online
Ethnologue catalog an unfortunate current tendency for NLP algorithms
to be developed or tested just


In [29]:
chapters = [
  "Introduction",
  "Regular Expressions, Tokenization, Edit Distance",
  "N-gram Language Models",
  "Naive Bayes, Text Classification, and Sentiment",
  "Logistic Regression",
  "Vector Semantics and Embeddings",
  "Neural Networks",
  "RNNs and LSTMs",
  "The Transformer",
  "Large Language Models",
  "Masked Language Models",
  "Model Alignment, Prompting, and In-Context Learning",
  "Machine Translation",
  "Question Answering, Information Retrieval, and RAG",
  "Chatbots & Dialogue Systems",
  "Automatic Speech Recognition and Text-to-Speech",
  "Annotating Linguistic Structure",
  "Sequence Labeling for Parts of Speech and Named Entities",
  "Context-Free Grammars and Constituency Parsing",
  "Dependency Parsing",
  "Information Extraction: Relations, Events, and Time",
  "Semantic Role Labeling",
  "Lexicons for Sentiment, Affect, and Connotation",
  "Coreference Resolution and Entity Linking",
  "Discourse Coherence",
]

In [30]:
!pip install fuzzywuzzy

from fuzzywuzzy import fuzz

matches = {}
for section in chapters :
    found = any(fuzz.partial_ratio(section.lower(), para.lower()) > 25 for para in paragraphs)

    #found = any(section.lower() in para.lower() for para in paragraphs)
    matches[section] = "✅ Found" if found else "❌ Not Found"

for k, v in matches.items():
    print(f"{k}: {v}")

Introduction: ✅ Found
Regular Expressions, Tokenization, Edit Distance: ✅ Found
N-gram Language Models: ✅ Found
Naive Bayes, Text Classification, and Sentiment: ✅ Found
Logistic Regression: ✅ Found
Vector Semantics and Embeddings: ✅ Found
Neural Networks: ✅ Found
RNNs and LSTMs: ✅ Found
The Transformer: ✅ Found
Large Language Models: ✅ Found
Masked Language Models: ✅ Found
Model Alignment, Prompting, and In-Context Learning: ✅ Found
Machine Translation: ✅ Found
Question Answering, Information Retrieval, and RAG: ✅ Found
Chatbots & Dialogue Systems: ✅ Found
Automatic Speech Recognition and Text-to-Speech: ✅ Found
Annotating Linguistic Structure: ✅ Found
Sequence Labeling for Parts of Speech and Named Entities: ✅ Found
Context-Free Grammars and Constituency Parsing: ✅ Found
Dependency Parsing: ✅ Found
Information Extraction: Relations, Events, and Time: ✅ Found
Semantic Role Labeling: ✅ Found
Lexicons for Sentiment, Affect, and Connotation: ✅ Found
Coreference Resolution and Entity Linki

In [31]:
# Install necessary libraries
!pip install transformers fuzzywuzzy sentencepiece --quiet

# Imports
import pandas as pd
import textwrap
import torch
from transformers import (
    BertTokenizerFast,
    T5Tokenizer,
    T5ForConditionalGeneration,
    pipeline
)
from fuzzywuzzy import fuzz
from tqdm import tqdm

In [32]:
# Load and clean data
df = pd.read_csv('/content/dataset.csv', sep='|')

paragraphs = df['text'].dropna().sample(n=1000, random_state=42).tolist()
paragraphs = list(set([p.strip() for p in paragraphs if len(p.strip()) > 0]))


In [33]:
'''# Optional: Deduplicate highly similar paragraphs
unique_paragraphs = []
seen = set()
for para in paragraphs:
    if all(fuzz.ratio(para, seen_para) < 90 for seen_para in seen):
        unique_paragraphs.append(para)
        seen.add(para)
paragraphs = unique_paragraphs'''

'# Optional: Deduplicate highly similar paragraphs\nunique_paragraphs = []\nseen = set()\nfor para in paragraphs:\n    if all(fuzz.ratio(para, seen_para) < 90 for seen_para in seen):\n        unique_paragraphs.append(para)\n        seen.add(para)\nparagraphs = unique_paragraphs'

In [34]:
# Tokenizer and chunking setup
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def chunk_paragraphs(paragraphs, tokenizer, max_length=512, stride=64):
    chunked = []
    for idx, para in enumerate(paragraphs):
        tokens = tokenizer(para,
                           return_overflowing_tokens=True,
                           truncation=True,
                           padding=True,
                           max_length=max_length,
                           stride=stride,
                           return_tensors='pt')
        for i in range(len(tokens['input_ids'])):
            chunked.append({
                "input_ids": tokens['input_ids'][i],
                "attention_mask": tokens['attention_mask'][i],
                "text": para
            })
    return chunked

chunked_data = chunk_paragraphs(paragraphs, tokenizer)
print(f"Total chunks: {len(chunked_data)}")

Total chunks: 1032


In [35]:
# Load T5 model for question generation
tokenizer_qg = T5Tokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model_qg = T5ForConditionalGeneration.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

def generate_question(context):
    input_text = f"generate question: {context}"
    inputs = tokenizer_qg.encode(input_text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model_qg.generate(inputs, max_length=64, num_beams=4, early_stopping=True)
    return tokenizer_qg.decode(outputs[0], skip_special_tokens=True)


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [36]:

# Load BERT QA pipeline
qa_pipeline = pipeline("question-answering", model="deepset/bert-base-cased-squad2")

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [37]:
# Generate QA pairs
automatic = []
sampled_chunks = chunked_data

for entry in tqdm(sampled_chunks):
    context = entry["text"]
    question = generate_question(context)
    if question:
        try:
            answer = qa_pipeline(question=question, context=context)
            automatic.append({
                "question": question,
                "answer": answer.get("answer", ""),
                "context": context
            })
        except:
            continue

# Save results
df_qa = pd.DataFrame(automatic)
df_qa.to_csv("automatic.csv", index=False)
print("Saved to automatic.csv")


 48%|████▊     | 495/1032 [34:25<46:40,  5.22s/it]/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
100%|██████████| 1032/1032 [1:10:30<00:00,  4.10s/it]

Saved to automatic.csv


In [38]:
import pandas as pd
df = pd.read_csv('/content/automatic.csv')
df

,question,answer,context
0,question: What does the Penn Treebank tag?,tagset,the Penn Treebank tagset:
1,question: What is the target language sentence?,an interlingua,"In the early years, the space of MT architectu..."
2,question: What would have happened if X and w ...,y,Note by the way that we could have kept X and ...
3,question: What is the bias term b?,the weighted sum of the evidence for the class,To make a decision on a test instance—after we...
4,"question: What is the function L(a,b,c) = c(a+...",component addition and multiplication,"Consider computing the function L(a,b,c) = c(a..."
...,...,...,...
1027,question: What do SaFeRDialogues do?,Taking feedback gracefully after conversationa...,SaFeRDialogues: Taking feedback gracefully aft...
1028,question: How does word2vec simplify classific...,training a logistic regression classiﬁer,We’ll see how to do neural networks in the nex...
1029,question: What is the task of the classifier f...,classiﬁer,Let’s consider the task of this classiﬁer for ...
1030,question: What does the dummy node a0 do?,1,Replacing the bias unit In describing networks...


In [39]:


import pandas as pd

manual_df = pd.read_csv("manual.csv", sep=",")

#Concatenate automatic QA files into one
automatic_combined_df = pd.read_csv("automatic.csv")

#Combine manual and automatic QA pairs into one
combined_df = pd.concat([manual_df, automatic_combined_df], ignore_index=True)

#Shuffle the combined dataset
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

#Save the combined dataset to a CSV
combined_df.to_csv("combined_qa.csv", sep="|", index=False, encoding="utf-8")


print("Combined QA pairs saved to 'combined_qa.csv'")


Combined QA pairs saved to 'combined_qa.csv'


In [40]:
import pandas as pd
df = pd.read_csv('/content/combined_qa.csv',sep = '|')
df

,question,answer,context
0,question: What does ACL stand for?,"mutual information, and lexicography","Word association norms, mutual information, an..."
1,question: How can RNN-based language models be...,question answering,Generation with RNN-Based Language Models RNN-...
2,question: What type of language is deterministic?,no probabilities,language that is deterministic (no probabiliti...
3,What is retrieval-augmented generation in the ...,Retrieval-augmented generation involves using ...,NaN
4,question: What is one example of a generative ...,56 language model,"fragment of word, 13 Gaussian prior on weights..."
...,...,...,...
1186,question: What is the name of the website that...,org,www.geonames.org www.census.gov preﬁx(xi) = L ...
1187,question: Data elements and what else?,Data elements,Data elements and
1188,question: What does parallelizing multi-head a...,the input and output have the model dimension d,Parallelizing multi-head attention In multi-he...
1189,question: What is a fixed-length context vector?,ci is one of the inputs to the computation of ...,"αij he With this, we ﬁnally have a ﬁxed-length..."


In [41]:
import pandas as pd

# Load your automatic QA file
automatic_df = pd.read_csv("automatic.csv")

# Clean question column if needed
automatic_df["question"] = automatic_df["question"].str.replace(r"^question:\s*", "", regex=True).str.strip()

# Sample 10 random QA pairs
sampled_auto = automatic_df.sample(n=10, random_state=42)[["question", "answer", "context"]]

# Save to CSV for manual evaluation
sampled_auto.to_csv("manual_rating.csv", index=False)
print("Saved to manual_rating.csv")


Saved to manual_rating.csv


In [42]:
import pandas as pd
df = pd.read_csv('/content/manual_rating.csv',sep = '|')
df

,"question,answer,context"
0,What is Semisupervised Relation Extraction via...
1,"Are TTS systems speaker-dependent?,trained to ..."
2,"What is the first step of decoding?,we compute..."
3,"Who is J. Nivre?,A. Meyers,""Hajiˇc, J., M. Cia..."
4,Step Stack Word List Action Relation Added wha...
5,What is the target text for Direct Translation...
6,Tensor product variable binding and the repres...
7,What is the centroid of the embeddings of each...
8,"Who is Kocijan?,""Kocijan,"",""Kocijan,"""
9,What is the meaning of a word in a dictionary?...


In [43]:
import pandas as pd
import json
import uuid

# Load manual QA CSV
df = pd.read_csv("manual.csv")

# Convert to SQuAD-style JSON (using dummy context based on the answer)
squad_data = {
    "version": "v2.0",
    "data": [
        {
            "title": "ManualQA",
            "paragraphs": []
        }
    ]
}

for _, row in df.iterrows():
    question = row["question"]
    answer = row["answer"]
    context = f"{answer} is the correct answer."

    answer_start = context.lower().find(answer.lower())
    if answer_start == -1:
        continue  # skip if answer isn't in context

    qa = {
        "id": str(uuid.uuid4()),
        "question": question,
        "answers": [{"text": answer, "answer_start": answer_start}],
        "is_impossible": False
    }

    squad_data["data"][0]["paragraphs"].append({
        "context": context,
        "qas": [qa]
    })

# Save as JSON
with open("manual_qa_squad.json", "w") as f:
    json.dump(squad_data, f, indent=2)

print("Saved to manual_qa_squad.json")


Saved to manual_qa_squad.json


In [44]:
import pandas as pd
import json
import uuid

# Load automatic QA CSV
df = pd.read_csv("automatic.csv")

# Clean question
df["question"] = df["question"].str.replace(r"^question:\s*", "", regex=True).str.strip()
df["answer"] = df["answer"].str.lower().str.strip()

# Convert to SQuAD format
squad_data = {
    "version": "v2.0",
    "data": [
        {
            "title": "AutomaticQA",
            "paragraphs": []
        }
    ]
}

for _, row in df.iterrows():
    question = row["question"]
    answer = row["answer"]
    context = row["context"]

    answer_start = context.lower().find(answer.lower())
    if answer_start == -1:
        continue  # skip if answer not in context

    qa = {
        "id": str(uuid.uuid4()),
        "question": question,
        "answers": [{"text": answer, "answer_start": answer_start}],
        "is_impossible": False
    }

    squad_data["data"][0]["paragraphs"].append({
        "context": context,
        "qas": [qa]
    })

# Save as JSON
with open("automatic_qa_squad.json", "w") as f:
    json.dump(squad_data, f, indent=2)

print("Saved to automatic_qa_squad.json")


Saved to automatic_qa_squad.json


In [45]:
import json

# Load both files
with open("manual_qa_squad.json") as f:
    manual_data = json.load(f)

with open("automatic_qa_squad.json") as f:
    auto_data = json.load(f)

# Combine
combined_data = {
    "version": "v2.0",
    "data": manual_data["data"] + auto_data["data"]
}

# Save
with open("combined_qa_squad.json", "w") as f:
    json.dump(combined_data, f, indent=2)

print("Saved to combined_qa_squad.json")


Saved to combined_qa_squad.json


In [46]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [47]:
import json

# Load your original dataset
with open('/content/combined_qa_squad.json') as f:
    data = json.load(f)

# Prepare a new dataset structure
squad_data = []

for item in data['data']:
    title = item['title']
    for paragraph in item['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            answers = qa['answers'][0]['text']
            answer_start = qa['answers'][0]['answer_start']

            # Format each entry as a SQuAD-style QA pair
            squad_data.append({
                'context': context,
                'question': question,
                'answers': {
                    'text': [answers],
                    'answer_start': [answer_start]
                }
            })

# Save the restructured data to a new JSON file
with open('/content/formatted_dataset.json', 'w') as f:
    json.dump(squad_data, f, indent=4)


In [48]:
'''from datasets import load_dataset

# Load the newly reformatted dataset
dataset = load_dataset('json', data_files={'train': '/content/formatted_dataset.json' })

# Inspect the dataset
print(dataset['train'][0])  # Check the first item in the training set
'''

"from datasets import load_dataset\n\n# Load the newly reformatted dataset\ndataset = load_dataset('json', data_files={'train': '/content/formatted_dataset.json' })\n\n# Inspect the dataset\nprint(dataset['train'][0])  # Check the first item in the training set\n"

In [49]:
from datasets import load_dataset

# Load the full dataset as a single set
full_dataset = load_dataset('json', data_files='/content/formatted_dataset.json', split='train')

# Split into 80% train and 20% validation
split_dataset = full_dataset.train_test_split(test_size=0.2, seed=42)

# Access the splits
train_dataset = split_dataset['train']
valid_dataset = split_dataset['test']

# Check one example from each
print("Train sample:", train_dataset[0])
print("Validation sample:", valid_dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

Train sample: {'context': 'Processing many examples at once We’ve shown the equations for logistic regression for a single example. But in practice we’ll of course want to process an entire test set with many examples. Let’s suppose we have a test set consisting of m test examples each of which we’d like to classify. We’ll continue to use the notation from page 78, in which a superscript value in parentheses refers to the example index in some set of data (either for training or for test). So in this case each test example x(i) has a feature vector x(i), 1 ≤i ≤m. (As usual, we’ll represent vectors and matrices in bold.) One way to compute each output value ˆy(i) is just to have a for-loop, and compute each test example one at a time:', 'question': 'How can we process an entire test set with many examples?', 'answers': {'answer_start': [33], 'text': ['we’ve shown the equations for logistic regression']}}
Validation sample: {'context': 'Example of the use of chain-of-thought prompting (r

In [50]:
#Preprocess the Combined Dataset

from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

# Tokenization and preprocessing function
def tokenize_function(examples):
    return tokenizer(examples['question'], examples['context'], truncation=True, padding='max_length', max_length=512)

# Apply the tokenization function
#tokenized_datasets = dataset.map(tokenize_function, batched=True)


# Apply the tokenization function to both train and validation datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/952 [00:00<?, ? examples/s]

Map:   0%|          | 0/239 [00:00<?, ? examples/s]

In [61]:
print(train_dataset.column_names)
print(valid_dataset.column_names)


['context', 'question', 'answers', 'input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions']
['context', 'question', 'answers', 'input_ids', 'token_type_ids', 'attention_mask']


In [62]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

# Define Labels for Training (start and end positions)
def positions(example, is_train = True):
    #print(example)
    # Tokenize the question and context with truncation and padding to max_length
    inputs = tokenizer(example['question'], example['context'], truncation=True, padding='max_length', max_length=512, return_offsets_mapping=True)

    # Get the offset mapping and remove it from inputs
    offset_mapping = inputs.pop("offset_mapping")
    input_ids = inputs["input_ids"]
    sequence_ids = inputs.sequence_ids()

    # Ensure sequence_ids has the same length as input_ids by extending it if necessary
    sequence_ids += [1] * (len(input_ids) - len(sequence_ids))

    # Answer information
    answer = example["answers"]["text"][0]
    start_char = example["answers"]["answer_start"][0]

    # Ensure start_char is not a list and handle it correctly
    if isinstance(start_char, list):
        start_char = start_char[0]
    end_char = start_char + len(answer)

    # Find the token indices for start and end positions
    start_position, end_position = None, None

    # Loop over each token's offset in the offset_mapping
    for idx, token_offsets in enumerate(offset_mapping):
        if len(token_offsets) != 2:
            #print(f"Skipping invalid entry in offset_mapping at index {idx}: {token_offsets}")
            continue

        start, end = token_offsets  # Unpack the tuple

        # Debugging: Check if the current token is overlapping with the answer
        if start <= start_char < end:
            #print(f"Token {idx} covers answer start at char {start_char} (start: {start}, end: {end})")
            start_position = idx
        if start < end_char <= end:
            #print(f"Token {idx} covers answer end at char {end_char} (start: {start}, end: {end})")
            end_position = idx
            break

    # If no positions were found and it is for training, handle the error
    if is_train and (start_position is None or end_position is None):
        return None  # Skip this example in the training dataset

    # Add start and end positions for training, but not for validation
    if is_train:
        inputs["start_positions"] = start_position
        inputs["end_positions"] = end_position

    return inputs

# Apply the positions function to both train and validation datasets
train_dataset = train_dataset.map(lambda example: positions(example, is_train=True), batched=False)
valid_dataset = valid_dataset.map(lambda example: positions(example, is_train=False), batched=False)
# Check the tokenized dataset
print(train_dataset[0])
print(valid_dataset[0])

    # If no positions were found, handle the error more gracefully
'''if start_position is None or end_position is None:
        #print(f"Could not find answer positions for question: {example['question']}")
        return None  # Skip this example

    # Return the tokenized inputs with the added start and end positions
    inputs["start_positions"] = start_position
    inputs["end_positions"] = end_position
    return inputs

# Apply the positions function to both train and validation datasets
train_dataset = train_dataset.map(positions, batched=False)
valid_dataset = valid_dataset.map(positions, batched=False)'''

# Check the tokenized dataset
#print(train_dataset[0])

#print(valid_dataset[0])




Map:   0%|          | 0/893 [00:00<?, ? examples/s]

Map:   0%|          | 0/239 [00:00<?, ? examples/s]

{'context': 'Processing many examples at once We’ve shown the equations for logistic regression for a single example. But in practice we’ll of course want to process an entire test set with many examples. Let’s suppose we have a test set consisting of m test examples each of which we’d like to classify. We’ll continue to use the notation from page 78, in which a superscript value in parentheses refers to the example index in some set of data (either for training or for test). So in this case each test example x(i) has a feature vector x(i), 1 ≤i ≤m. (As usual, we’ll represent vectors and matrices in bold.) One way to compute each output value ˆy(i) is just to have a for-loop, and compute each test example one at a time:', 'question': 'How can we process an entire test set with many examples?', 'answers': {'answer_start': [33], 'text': ['we’ve shown the equations for logistic regression']}, 'input_ids': [101, 1731, 1169, 1195, 1965, 1126, 2072, 2774, 1383, 1114, 1242, 5136, 136, 102, 18

'if start_position is None or end_position is None:\n        #print(f"Could not find answer positions for question: {example[\'question\']}")\n        return None  # Skip this example\n\n    # Return the tokenized inputs with the added start and end positions\n    inputs["start_positions"] = start_position\n    inputs["end_positions"] = end_position\n    return inputs\n\n# Apply the positions function to both train and validation datasets\ntrain_dataset = train_dataset.map(positions, batched=False)\nvalid_dataset = valid_dataset.map(positions, batched=False)'

In [53]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

# Define Labels for Training (start and end positions)
def positions(example, is_train=True):
    # Tokenize the question and context with truncation and padding to max_length
    inputs = tokenizer(example['question'], example['context'], truncation=True, padding='max_length', max_length=512, return_offsets_mapping=True)

    # Get the offset mapping and remove it from inputs
    offset_mapping = inputs.pop("offset_mapping")
    input_ids = inputs["input_ids"]
    sequence_ids = inputs.sequence_ids()

    # Ensure sequence_ids has the same length as input_ids by extending it if necessary
    sequence_ids += [1] * (len(input_ids) - len(sequence_ids))

    # Answer information
    answer = example["answers"]["text"][0]
    start_char = example["answers"]["answer_start"][0]

    # Ensure start_char is not a list and handle it correctly
    if isinstance(start_char, list):
        start_char = start_char[0]
    end_char = start_char + len(answer)

    # Find the token indices for start and end positions
    start_position, end_position = None, None

    # Loop over each token's offset in the offset_mapping
    for idx, token_offsets in enumerate(offset_mapping):
        if len(token_offsets) != 2:
            continue

        start, end = token_offsets  # Unpack the tuple

        if start <= start_char < end:
            start_position = idx
        if start < end_char <= end:
            end_position = idx
            break

    # If no positions were found, set to default (-1) instead of returning None
    if is_train and (start_position is None or end_position is None):
        #print(f"Answer not found for question: {example['question']}")  # Debugging line
        start_position = end_position = -1  # Default invalid position for training

    # Add start and end positions for training, but not for validation
    if is_train:
        inputs["start_positions"] = start_position
        inputs["end_positions"] = end_position

    return inputs

# Check the original datasets before applying the map function
print("Original training dataset sample:", train_dataset[0])  # Check a sample from the train dataset
print("Original validation dataset sample:", valid_dataset[0])  # Check a sample from the validation dataset

# Apply the positions function to both train and validation datasets
train_dataset = train_dataset.map(lambda example: positions(example, is_train=True), batched=False)
valid_dataset = valid_dataset.map(lambda example: positions(example, is_train=False), batched=False)

# Check the tokenized datasets after processing
print("Tokenized training dataset sample:", train_dataset[0])  # Check a sample from the train dataset
print("Tokenized validation dataset sample:", valid_dataset[0])  # Check a sample from the validation dataset


Original training dataset sample: {'context': 'Processing many examples at once We’ve shown the equations for logistic regression for a single example. But in practice we’ll of course want to process an entire test set with many examples. Let’s suppose we have a test set consisting of m test examples each of which we’d like to classify. We’ll continue to use the notation from page 78, in which a superscript value in parentheses refers to the example index in some set of data (either for training or for test). So in this case each test example x(i) has a feature vector x(i), 1 ≤i ≤m. (As usual, we’ll represent vectors and matrices in bold.) One way to compute each output value ˆy(i) is just to have a for-loop, and compute each test example one at a time:', 'question': 'How can we process an entire test set with many examples?', 'answers': {'answer_start': [33], 'text': ['we’ve shown the equations for logistic regression']}, 'input_ids': [101, 1731, 1169, 1195, 1965, 1126, 2072, 2774, 13

Map:   0%|          | 0/893 [00:00<?, ? examples/s]

Map:   0%|          | 0/239 [00:00<?, ? examples/s]

Tokenized training dataset sample: {'context': 'Processing many examples at once We’ve shown the equations for logistic regression for a single example. But in practice we’ll of course want to process an entire test set with many examples. Let’s suppose we have a test set consisting of m test examples each of which we’d like to classify. We’ll continue to use the notation from page 78, in which a superscript value in parentheses refers to the example index in some set of data (either for training or for test). So in this case each test example x(i) has a feature vector x(i), 1 ≤i ≤m. (As usual, we’ll represent vectors and matrices in bold.) One way to compute each output value ˆy(i) is just to have a for-loop, and compute each test example one at a time:', 'question': 'How can we process an entire test set with many examples?', 'answers': {'answer_start': [33], 'text': ['we’ve shown the equations for logistic regression']}, 'input_ids': [101, 1731, 1169, 1195, 1965, 1126, 2072, 2774, 1

In [54]:
print(train_dataset.column_names)
print(valid_dataset.column_names)


['context', 'question', 'answers', 'input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions']
['context', 'question', 'answers', 'input_ids', 'token_type_ids', 'attention_mask']


In [55]:

from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained("bert-base-cased")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
#!pip install --upgrade transformers


In [57]:
import transformers
print(transformers.__version__)


4.51.3


In [58]:
import datasets
print(datasets.__version__)


3.5.1


In [59]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [60]:
from transformers import Trainer, TrainingArguments, BertForQuestionAnswering, BertTokenizerFast

# Model and tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-base-cased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="steps",     # Save at specific steps if you want
    save_steps=500,            # Save model every 500 steps
    learning_rate=2e-5,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    weight_decay=0.01,
    logging_dir="./logs",      # Log directory
)


# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=None
)

# Start training
trainer.train()


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-60-d0e01c506487>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=224, training_loss=3.063128335135324, metrics={'train_runtime': 11517.9062, 'train_samples_per_second': 0.155, 'train_steps_per_second': 0.019, 'total_flos': 466676007530496.0, 'train_loss': 3.063128335135324, 'epoch': 2.0})

In [63]:
# Save model
model_path = "./model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)


('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json',
 './model/tokenizer.json')

In [64]:
#Load and Test the Model

from transformers import pipeline

qa_pipeline = pipeline("question-answering", model=model_path, tokenizer=model_path)

context = "Transformers provide state-of-the-art natural language processing tools."
question = "What do transformers provide?"

result = qa_pipeline({"context": context, "question": question})
print(result['answer'])


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Transformers provide state-of-the-art natural language processing tools


In [65]:
import gradio as gr

def answer_question(context, question):
    result = qa_pipeline({"context": context, "question": question})
    return result['answer']

interface = gr.Interface(
    fn=answer_question,
    inputs=["text", "text"],
    outputs="text",
    title="BERT Question Answering",
    description="Ask questions based on a given context."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed28fb30718d3b17ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
